In [ ]:
import os
import queue
import threading
import time
from google.cloud import speech
import pyaudio

# Audio recording parameters
RATE = 16000
CHUNK = int(RATE / 10)  # 100ms chunks
FORMAT = pyaudio.paInt16
CHANNELS = 1

def main():
    # Create a thread-safe queue for the audio data
    audio_queue = queue.Queue()
    
    # Create a flag to indicate if we should stop
    stop_event = threading.Event()
    
    # Define a function to capture audio from the microphone
    def audio_capture_thread():
        print("🎙️ Starting microphone... (Speak now)")
        p = pyaudio.PyAudio()
        stream = p.open(
            format=FORMAT,
            channels=CHANNELS,
            rate=RATE,
            input=True,
            frames_per_buffer=CHUNK
        )
        
        while not stop_event.is_set():
            data = stream.read(CHUNK, exception_on_overflow=False)
            audio_queue.put(data)
        
        # Cleanup
        stream.stop_stream()
        stream.close()
        p.terminate()
        print("🛑 Microphone stopped")
    
    # Start the audio capture thread
    capture_thread = threading.Thread(target=audio_capture_thread)
    capture_thread.daemon = True
    capture_thread.start()
    
    # Create a Speech client
    client = speech.SpeechClient()
    
    # Configure the recognition settings
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=RATE,
        language_code="en-US",
        model="default",  # Change to "medical_conversation" if needed
        enable_automatic_punctuation=True,
    )
    
    streaming_config = speech.StreamingRecognitionConfig(
        config=config,
        interim_results=True,  # Show interim results
    )
    
    # Function to generate requests for the streaming API
    def generate_requests():
        while not stop_event.is_set():
            # Use a timeout to avoid blocking indefinitely
            try:
                chunk = audio_queue.get(block=True, timeout=0.5)
                yield speech.StreamingRecognizeRequest(audio_content=chunk)
            except queue.Empty:
                continue
    
    try:
        print("⚙️ Starting speech recognition...")
        requests = generate_requests()
        responses = client.streaming_recognize(streaming_config, requests)
        
        # Process the responses
        for response in responses:
            if not response.results:
                continue
            
            result = response.results[0]
            is_final = result.is_final
            transcript = result.alternatives[0].transcript
            
            # Clear the line and print the transcript
            print(f"\r{'FINAL: ' if is_final else 'Interim: '}{transcript}", end="", flush=True)
            
            # Add a newline when we get a final result
            if is_final:
                print()
        
    except KeyboardInterrupt:
        print("\n👋 Stopping by user request")
    except Exception as e:
        print(f"\n❌ Error: {e}")
    finally:
        # Cleanup
        stop_event.set()
        capture_thread.join(timeout=2.0)

if __name__ == "__main__":
    print("Real-time Speech Recognition Test")
    print("--------------------------------")
    print("This script will capture audio from your microphone and")
    print("display real-time transcription results.")
    print("Press Ctrl+C to stop.")
    print()
    
    # Check if Google credentials are set
    if not os.environ.get("GOOGLE_APPLICATION_CREDENTIALS"):
        print("⚠️  Warning: GOOGLE_APPLICATION_CREDENTIALS environment variable not set.")
        print("    You may need to run:")
        print('    export GOOGLE_APPLICATION_CREDENTIALS="path/to/your-key.json"')
        print()
    
    main()